<a href="https://colab.research.google.com/github/sugarlin1732/NLP_Transformer_practice/blob/master/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import lib
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output
import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = ".//gdrive//My Drive//Transformer_practice//"
if not os.path.isdir(path):
    os.mkdir(path)


In [0]:
# ignore unnecessary warnings
import logging
logging.basicConfig(level=logging.ERROR)

np.set_printoptions(suppress=True) # disable scientific notation

In [0]:
# download training data ("wmt19_translate")
# make download dir
download_dir = ".//download_dir"
if not os.path.isdir(download_dir):
    os.mkdir(download_dir)

config = tfds.translate.wmt.WmtConfig(
    version="0.0.1",
    language_pair=("zh", "en"), # zh-en translation
    subsets={
        tfds.Split.TRAIN: ["newscommentary_v14"]
    },
)
builder = tfds.builder("wmt_translate", config=config)

builder.download_and_prepare(download_dir=download_dir)
clear_output()


In [0]:
# dataset split

training_set, val_set, testing_set = builder.as_dataset(split=\
    ("train[:50%]", "train[50%:75%]", "train[75%:]") \
    , as_supervised=True)
print(training_set)
print(val_set)
print(testing_set)

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>
<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>
<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.string)>


In [0]:
# check data
for en, zh in training_set.take(2):
  print(en)
  print(zh)
  print('-' * 10)

tf.Tensor(b'The fear is real and visceral, and politicians ignore it at their peril.', shape=(), dtype=string)
tf.Tensor(b'\xe8\xbf\x99\xe7\xa7\x8d\xe6\x81\x90\xe6\x83\xa7\xe6\x98\xaf\xe7\x9c\x9f\xe5\xae\x9e\xe8\x80\x8c\xe5\x86\x85\xe5\x9c\xa8\xe7\x9a\x84\xe3\x80\x82 \xe5\xbf\xbd\xe8\xa7\x86\xe5\xae\x83\xe7\x9a\x84\xe6\x94\xbf\xe6\xb2\xbb\xe5\xae\xb6\xe4\xbb\xac\xe5\x89\x8d\xe9\x80\x94\xe5\xa0\xaa\xe5\xbf\xa7\xe3\x80\x82', shape=(), dtype=string)
----------
tf.Tensor(b'In fact, the German political landscape needs nothing more than a truly liberal party, in the US sense of the word \xe2\x80\x9cliberal\xe2\x80\x9d \xe2\x80\x93 a champion of the cause of individual freedom.', shape=(), dtype=string)
tf.Tensor(b'\xe4\xba\x8b\xe5\xae\x9e\xe4\xb8\x8a\xef\xbc\x8c\xe5\xbe\xb7\xe5\x9b\xbd\xe6\x94\xbf\xe6\xb2\xbb\xe5\xb1\x80\xe5\x8a\xbf\xe9\x9c\x80\xe8\xa6\x81\xe7\x9a\x84\xe4\xb8\x8d\xe8\xbf\x87\xe6\x98\xaf\xe4\xb8\x80\xe4\xb8\xaa\xe7\xac\xa6\xe5\x90\x88\xe7\xbe\x8e\xe5\x9b\xbd\xe6\x89\x80\xe8\

In [0]:
# numpy decode the coding of examples and save as samples
samples = []
sample_num = 10
for en_raw, zh_raw in training_set.take(sample_num):
    en = en_raw.numpy().decode("utf-8")
    zh = zh_raw.numpy().decode("utf-8")
    print(en)
    print(zh)
    samples.append((en, zh))

The fear is real and visceral, and politicians ignore it at their peril.
这种恐惧是真实而内在的。 忽视它的政治家们前途堪忧。
In fact, the German political landscape needs nothing more than a truly liberal party, in the US sense of the word “liberal” – a champion of the cause of individual freedom.
事实上，德国政治局势需要的不过是一个符合美国所谓“自由”定义的真正的自由党派，也就是个人自由事业的倡导者。
Shifting to renewable-energy sources will require enormous effort and major infrastructure investment.
必须付出巨大的努力和基础设施投资才能完成向可再生能源的过渡。
In this sense, it is critical to recognize the fundamental difference between “urban villages” and their rural counterparts.
在这方面，关键在于认识到“城市村落”和农村村落之间的根本区别。
A strong European voice, such as Nicolas Sarkozy’s during the French presidency of the EU, may make a difference, but only for six months, and at the cost of reinforcing other European countries’ nationalist feelings in reaction to the expression of “Gallic pride.”
法国担任轮值主席国期间尼古拉·萨科奇统一的欧洲声音可能让人耳目一新，但这种声音却只持续了短短六个月，而且付出了让其他欧洲国家在面对“高卢人的骄傲”时民族主义情感进一步被激发的代价。
Most of Japan’s bondhold

In [0]:
# make dictionary
# use SubwordTextEncoder to convert word into numbers
%%time
# make en dictionary
try:
    # load existed dictionary if exists
    subword_en = tfds.features.text.SubwordTextEncoder.load_from_file( \
                path + "en_dictionary")
except:
    subword_en = tfds.features.text.SubwordTextEncoder.build_from_corpus( \
                (en_word.numpy() for en_word, zh_word in training_set), \
                target_vocab_size = 2**15)
                # en_word.numpy() no need to decode
    subword_en.save_to_file(path + "en_dictionary") # save dictionary
print(f"英文字典大小：{subword_en.vocab_size}")
print(f"前 10 個英文  subwords：{subword_en.subwords[:10]}")

# make zh dictionary
try:
    subword_zh = tfds.features.text.SubwordTextEncoder.load_from_file( \
                path + "zh_dictionary")
except:
    subword_zh = tfds.features.text.SubwordTextEncoder.build_from_corpus( \
                (zh_word.numpy() for en_word, zh_word in training_set), \
                target_vocab_size = 2**15)
                # zh_word.numpy() no need to decode
    subword_zh.save_to_file(path + "zh_dictionary") # save dictionary


print(f"中文字典大小：{subword_zh.vocab_size}")
print(f"前 10 個中文  subwords：{subword_zh.subwords[:10]}")


英文字典大小：32007
前 10 個英文  subwords：[', ', 'the_', 'to_', 'of_', 'and_', 'in_', 'a_', 'that_', 'is_', 's_']
中文字典大小：32061
前 10 個中文  subwords：['，', '。', '、', ' — — ', '（', '“', '的', '）', '”', '和']
CPU times: user 295 ms, sys: 7.15 ms, total: 302 ms
Wall time: 721 ms


In [0]:
# add BOS and EOS into sentence
# 加入起始以及結束token，編碼定義為字典最後加兩碼(subword_en.vocab_size, subword_en.vocab_size+1)

def add_BEOS(en_sentence, zh_sentence):
    en_tmp = [subword_en.vocab_size] + subword_zh.encode(en_sentence.numpy()) \
    + [subword_en.vocab_size+1]
    zh_tmp = [subword_zh.vocab_size] + subword_zh.encode(zh_sentence.numpy()) \
    + [subword_zh.vocab_size+1]

    return en_tmp, zh_tmp

# tensorflow 為靜態計算圖，需透過eager mode的api讓python直譯器可以直接和tf運算核心互動
def tf_add_BEOS(en_t, zh_t):
    return tf.py_function(add_BEOS, [en_t, zh_t], [tf.int64, tf.int64])

tmp_dataset = training_set.map(tf_add_BEOS)



In [0]:
# 訓練時每一組batch內的資料長度要一樣，要將batch裡的所有序列都用padded_batch到同樣長度

BATCH_SIZE = 128

tmp_dataset = tmp_dataset.padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
en_batch, zh_batch = next(iter(tmp_dataset))
print("英文索引序列的 batch")
print(en_batch)
print('-' * 20)
print("中文索引序列的 batch")
print(zh_batch)

英文索引序列的 batch
tf.Tensor(
[[32007  4547  6859 ...     0     0     0]
 [32007  7943 31837 ...     0     0     0]
 [32007 24844 14087 ...     0     0     0]
 ...
 [32007 16445 31837 ...     0     0     0]
 [32007  4547  6112 ...     0     0     0]
 [32007 19682  2846 ...     0     0     0]], shape=(128, 161), dtype=int64)
--------------------
中文索引序列的 batch
tf.Tensor(
[[32061   146  2230 ...     0     0     0]
 [32061    44     1 ...     0     0     0]
 [32061   125  2545 ...     0     0     0]
 ...
 [32061    28     1 ...     0     0     0]
 [32061  1363 21783 ...     0     0     0]
 [32061   659  8696 ...     0     0     0]], shape=(128, 56), dtype=int64)


In [0]:
#############  以上實驗  以下正式轉換  #########################################

In [0]:
# 正式將文字data一步轉換成訓練格式
batch_size = 128
shuffle_size = 15000

training_dataset = (
    training_set.map(tf_add_BEOS).cache().shuffle(shuffle_size)
    .padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
    .prefetch(tf.data.experimental.AUTOTUNE)
)

val_dataset = (
    val_set.map(tf_add_BEOS)
    .padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
)


# 資料以一組一組的batch(size=128)儲存
# check
# en_batch, zh_batch = next(iter(training_dataset))
# print("英文索引序列的 batch")
# print(en_batch)
# print('-' * 20)
# print("中文索引序列的 batch")
# print(zh_batch)


'\n# 資料以一組一組的batch(size=128)儲存\n# check\nen_batch, zh_batch = next(iter(training_dataset))\nprint("英文索引序列的 batch")\nprint(en_batch)\nprint(\'-\' * 20)\nprint("中文索引序列的 batch")\nprint(zh_batch)\n'

In [0]:
################################################################################

In [0]:
# 測試資料
demo_examples = [
    ("It is important.", "这很重要。"),
    ("The numbers speak for themselves.", "数字证明了一切。"),
]
pprint(demo_examples)

[('It is important.', '这很重要。'),
 ('The numbers speak for themselves.', '数字证明了一切。')]


In [0]:
# 將測試資料建成tf dataset
demo_examples = tf.data.Dataset.from_tensor_slices((
    [en for en, zh in demo_examples], [zh for en, zh in demo_examples]
))

try_data = demo_examples.map(tf_add_BEOS).padded_batch(2, padded_shapes=([-1], [-1]))

en_try, zh_try = next(iter(try_data))
print('en_try:', en_try)
print('' * 10)
print('zh_try:', zh_try)


en_try: tf.Tensor(
[[32007 31878  2132 12018  6514 23570 19665 31921 31851 32008     0     0
      0     0     0     0]
 [32007  4547 24819 12017 11163 31837  9085 13718  7465  3051  6185  6358
  27766 18206 31851 32008]], shape=(2, 16), dtype=int64)

zh_try: tf.Tensor(
[[32061 12332   346     2 32062     0]
 [32061   770  4435  1078     2 32062]], shape=(2, 6), dtype=int64)


In [0]:
# word embedding
en_dic_length = subword_en.vocab_size + 2 # 加上起始和結束的tokens
zh_dic_length = subword_zh.vocab_size + 2

# set embedding dimension
dimen = 4
embedding_layer_en = tf.keras.layers.Embedding(en_dic_length, dimen)
embedding_layer_zh = tf.keras.layers.Embedding(zh_dic_length, dimen)

emb_en_try = embedding_layer_en(en_try)
emb_zh_try = embedding_layer_zh(zh_try)
emb_en_try, emb_zh_try
# 輸出中每個list(batch)有兩句話，每句話有16(en)/6(zh)個字，每個字用4個維度表示

In [0]:
# padding mask
# 因為短的序列有補0，不希望訓練時的attention放在0上，需要padding mask把序列是0的地方遮蔽
# 概念類似讓序列是0的地方乘上很負的數，q和k點積完再經過softmax後的機率就會變得很低

def create_padding_mask(sequence):
    mask = tf.cast(tf.equal(sequence, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]
    #　為了後續broadcasting，把張量維度補到4維

en_try_mask = create_padding_mask(en_try)

In [0]:
############################### self-attention ###############################

In [0]:
# set random seed
tf.random.set_seed(9487)

q, k = emb_en_try, emb_en_try # self-attention的q和k一樣
v = tf.cast(tf.math.greater(tf.random.uniform(shape=emb_en_try.shape), 0.5),
    tf.float32) # 隨機產生跟q, k一樣shape的binary矩陣


In [0]:
# scaled dot product attention
def scaled_dot_product_attention(q, k, v, mask):
    # 將q, k做點積再scale
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)  # 取得 seq_k 的序列長度
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)  # scale by sqrt(dk)

    # 先將logits加上mask
    if mask is not None:
        scaled_attention_logits += (mask * -1e9) 
        # mask乘上極大負值，讓序列中0的地方變極負，softmax完趨近於0

    # 利用softmax計算總合為1的注意權重
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  
    # (..., seq_len_q, seq_len_k)
  
    # 以注意權重對v做加權平均（weighted average）
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [0]:
mask = en_try_mask
output, attention_weights = scaled_dot_product_attention(q, k, v, mask)
print("output:", output)
print("-" * 20)
print("attention_weights:", attention_weights)

output: tf.Tensor(
[[[[0.49994993 0.29993173 0.59999937 0.29995716]
   [0.49998304 0.29983938 0.6000892  0.30020246]
   [0.4999054  0.30020282 0.5996472  0.29974025]
   [0.49995193 0.3002391  0.59978414 0.29985556]
   [0.5000038  0.29993367 0.60008895 0.30000097]
   [0.4999317  0.3000367  0.5998912  0.29985514]
   [0.49994904 0.29974517 0.6000749  0.30022812]
   [0.4999966  0.2999573  0.60012144 0.2998963 ]
   [0.5000998  0.30003956 0.6001653  0.30007944]
   [0.5000279  0.2998821  0.6001103  0.3001626 ]
   [0.5000849  0.30025667 0.59992373 0.3001    ]
   [0.5000849  0.30025667 0.59992373 0.3001    ]
   [0.5000849  0.30025667 0.59992373 0.3001    ]
   [0.5000849  0.30025667 0.59992373 0.3001    ]
   [0.5000849  0.30025667 0.59992373 0.3001    ]
   [0.5000849  0.30025667 0.59992373 0.3001    ]]

  [[0.5000268  0.7000045  0.40006343 0.6999943 ]
   [0.50002754 0.6999756  0.40006247 0.70000076]
   [0.5000876  0.69979936 0.4001211  0.69987833]
   [0.50004154 0.6998825  0.39989197 0.6998811 ]